In [1]:
from pulp import *
import pulp
import pandas as pd
import numpy as np
import random

In [2]:
# Read the Excel file into a DataFrame
df = pd.read_excel('file.xlsx')

# Convert the DataFrame to a NumPy array
data = df.values
products = []

for row in data:
    predictedProfits = row[0]
    predictedProductQuantities = row[1]
    dimension = row[2]
    productName = row[3]

    product = {'predictedProfits': predictedProfits, 'predictedProductQuantities': predictedProductQuantities, 'dimension': dimension, 'productName': productName}
    products.append(product)

print(products)

[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1'}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2'}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3'}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4'}]


In [3]:
def calculate_profit(products):
    for product in products:
        profit = product['predictedProfits'] / product['predictedProductQuantities']
        product['productProfit'] = profit
    return products

In [4]:
productTest = calculate_profit(products)
print(productTest)

[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2', 'productProfit': 10.0}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3', 'productProfit': 10.0}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4', 'productProfit': 10.0}]


In [5]:
# Creation of the ILP model 
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)

In [6]:
# Define the decision variables
decisionVariables = []
def createDictionaryWithDecisionVariable():
    for product in products:
        decisionVariable = {product['productName']:  pulp.LpVariable(product['productName'], lowBound=0, cat='Integer')}
        decisionVariables.append(decisionVariable)
        
createDictionaryWithDecisionVariable()
print(decisionVariables)

[{'p1': p1}, {'p2': p2}, {'p3': p3}, {'p4': p4}]


In [7]:
# function to calculate objective function

def objectiveFunction(products):
    model = 0
    count = 0
    for product in products:
        model = model + (product['predictedProfits'] * decisionVariables[count][product['productName']])
        count = count + 1
    return model

#objective function
model += objectiveFunction(products)
print(model)

ILP_Problem:
MAXIMIZE
100*p1 + 200*p2 + 300*p3 + 10*p4 + 0
VARIABLES
0 <= p1 Integer
0 <= p2 Integer
0 <= p3 Integer
0 <= p4 Integer



In [5]:
#add user input for the multiple variables

minTotalAmount = int(input("Please insert minimum total amount of products: "))
maxTotalAmount = int(input("Please insert maximum  total amount of products: "))

#create two list with min and max amount per product by user insert

minAmountProduct = int(input("Please insert minimum  amount of products : "))
maxAmountProduct  = int(input("Please insert maximum  amount of products : "))

minProductVariety = int(input("Please insert minimum number of product variety: "))
maxProductVariety  = int(input("Please insert maximum  number of product variety: "))

storeDimension  = int(input("Please insert store dimension: "))

In [973]:
# Define the constraints

 # function to calculate variety of products
def get_variety(products):
    # Create an empty set to store the product names
    product_set = []
    
    # Iterate over the products
    for product in products:
        # Get the product name and add it to the set
        product_name = product['productName']
        product_set.append(product_name)
        
    # Return the length of the set
    return len(product_set)

print(get_variety(products))
variety=get_variety(products)

#see why it results in Infeasible
# model += variety <= maxProductVariety
# model += variety >= minProductVariety


#restriction to total amount of products

def maxMinTotalProductsRestriction(products):
    restrictFunction = 0
    count = 0
    for product in products:
        restrictFunction = restrictFunction + decisionVariables[count][product['productName']]
        count = count + 1
    return restrictFunction
    
model += maxMinTotalProductsRestriction(products) <= maxTotalAmount #restricao de quantidade maximo total de produtos
model += maxMinTotalProductsRestriction(products) >= minTotalAmount #restricao de valor minimo total de produtos

# variety of products

#restriction to maximum quantity of products
def maxMinQuantityProductsRestriction(products,model):
    count = 0
    for product in products:
        model += decisionVariables[count][product['productName']] <= maxAmountProduct
        model += decisionVariables[count][product['productName']] >= minAmountProduct
        count = count + 1
    
maxMinQuantityProductsRestriction(products, model)

print(maxAmountProduct)
print(minAmountProduct)


def maxDimensionProductsRestriction(products):
    count = 0
    restrictFunction = 0
    for product in products:                            
        restrictFunction = restrictFunction + (product['dimension'] * decisionVariables[count][product['productName']])
        count = count +1
    return restrictFunction

print(maxDimensionProductsRestriction(products))
#restricao de dimensões de produtos, menor que dimensao todal da loja
model += maxDimensionProductsRestriction(products) <= storeDimension 


4
10
1
5*p1 + 2*p2 + p3 + 20*p4


In [974]:
print(model)

ILP_Problem:
MAXIMIZE
100*p1 + 200*p2 + 300*p3 + 10*p4 + 0
SUBJECT TO
_C1: p1 + p2 + p3 + p4 <= 100

_C2: p1 + p2 + p3 + p4 >= 10

_C3: p1 <= 10

_C4: p1 >= 1

_C5: p2 <= 10

_C6: p2 >= 1

_C7: p3 <= 10

_C8: p3 >= 1

_C9: p4 <= 10

_C10: p4 >= 1

_C11: 5 p1 + 2 p2 + p3 + 20 p4 <= 1.11111111111e+21

VARIABLES
0 <= p1 Integer
0 <= p2 Integer
0 <= p3 Integer
0 <= p4 Integer



In [975]:
# Solve the problem
status = model.solve()

In [976]:
print(pulp.LpStatus[status])

Optimal


In [977]:
#print(pulp.value(p1), pulp.value(model.objective))

In [978]:
# Print the solution
#print("Solution:")
#print("x =", x.value())
#print("y =", y.value())

In [1069]:
def printValuesReturned():
    for v in model.variables():
        try:
            print(v.name,"=", v.value())
        except:
            print("error couldnt find value")
            
printValuesReturned()
print(len(products))
print(products)
print(products[0]['dimension'])

p1 = 10
p2 = 10
p3 = 10
p4 = 10
4
[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2', 'productProfit': 10.0}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3', 'productProfit': 10.0}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4', 'productProfit': 10.0}]
5


In [6]:
# Genetic Algorithm



productsList = []
print(productsList)
 
for product in products:
    productsList.append((product['productName'], product['predictedProductQuantities'], product['predictedProfits'], product['dimension']))
for p in productsList:
    print(p)

[]
('p1', 10, 100, 5)
('p2', 20, 200, 2)
('p3', 30, 300, 1)
('p4', 1, 10, 20)


In [10]:
population = []
populationSize=100
finalPopulation = []
taxNoCrossover = 10


# Functions to validate restrictions
# return 1 if restriction is repected else 0

#Total Amount Restriction
def validateTotalAmount(individuo):
    somaTotalAmountIndividuo = sum(individuo)
    if (somaTotalAmountIndividuo >= minTotalAmount) and (somaTotalAmountIndividuo <= maxTotalAmount):
        return 1
    else:
        return 0
    
#Total Quantity Restriction
def validateTotalQuantity(individuo):
    contador = 0
    individuoLen = len(individuo)
    for indAux in individuo:
        if (indAux >= minAmountProduct) and (indAux <= maxAmountProduct):
            contador = contador + 1
    if(contador == individuoLen):
        return 1
    else:
        return 0

#Dimension Restriction
def validateDimension(individuo, products):
    totalDimension = 0
    contador = 0
    for indAux in individuo:
        totalDimension = indAux * products[contador]['dimension']
        contador = contador + 1
    if(totalDimension > storeDimension):
        return 0
    else:
        return 1

In [11]:
# Create First Population

def firstpopulation(products):

    while len(population) < populationSize:
        solution = []
        totalQuantity = 0
        totalDimension = 0
        
        #generates the chromossomes each representing one product
        for j in range(len(products)):
            
            quantity = random.randint(0, maxAmountProduct)
            solution.append(quantity)
            
        #Total Amount Restriction
        totalAmountRestriction = validateTotalAmount(solution)
        #Total Quantity Restriction
        totalQuantityRestriction = validateTotalQuantity(solution)
        #Dimension Restriction
        dimensionRestriction = validateDimension(solution, products)
        
        if (totalAmountRestriction == 1) and (totalQuantityRestriction == 1) and (dimensionRestriction == 1):
            population.append(solution)
            

    print(len(firstpopulation(products)))
    print(population)
    
    return population



In [1127]:
firstpopulation(products)

print(len(firstpopulation(products)))
print(population)

100
[[4, 7, 7, 1], [4, 9, 10, 1], [9, 3, 3, 6], [6, 3, 2, 5], [10, 6, 1, 4], [7, 6, 7, 3], [8, 3, 9, 10], [1, 3, 9, 1], [6, 2, 3, 8], [1, 2, 9, 7], [6, 4, 4, 8], [7, 2, 7, 5], [4, 5, 8, 1], [5, 10, 8, 2], [8, 6, 7, 9], [9, 1, 10, 4], [2, 8, 9, 1], [8, 8, 1, 10], [5, 4, 8, 6], [6, 6, 1, 1], [4, 7, 5, 9], [9, 10, 7, 4], [8, 3, 2, 5], [1, 4, 10, 3], [7, 9, 8, 2], [4, 9, 2, 8], [5, 8, 9, 10], [9, 9, 5, 6], [3, 7, 4, 6], [10, 10, 5, 1], [1, 2, 3, 5], [4, 7, 4, 10], [8, 4, 6, 10], [6, 9, 10, 6], [5, 10, 6, 1], [4, 2, 9, 1], [9, 4, 3, 7], [7, 2, 6, 6], [3, 4, 8, 1], [7, 4, 5, 1], [6, 4, 10, 4], [6, 3, 5, 1], [10, 10, 4, 3], [10, 8, 1, 5], [3, 6, 4, 8], [4, 3, 8, 10], [7, 8, 7, 3], [7, 7, 1, 5], [5, 10, 8, 8], [8, 6, 2, 2], [7, 7, 9, 2], [10, 4, 9, 9], [6, 9, 1, 3], [9, 8, 9, 9], [2, 10, 1, 7], [2, 10, 7, 4], [6, 5, 1, 5], [5, 6, 5, 5], [5, 4, 8, 2], [9, 4, 1, 10], [8, 6, 4, 8], [4, 7, 9, 9], [6, 10, 1, 6], [9, 6, 6, 7], [5, 6, 9, 7], [6, 4, 4, 3], [3, 7, 9, 3], [7, 8, 9, 6], [5, 3, 3, 4], [5,

In [1110]:

arraySomaProfits = []
arrayFitProfitAndQuantity = []
finalPopulation = []

# if this element is 1 the first tax of best individuos is added to final population
elementToGetFirstBestIndividuos = 0

taxNoCrossover = 0.10

def createFitnes(population, products):
    
    for person in population:
        
        soma = 0
        contador = 0
        
        for productQuantity in person:
            
            soma = soma + (productQuantity * products[contador]['predictedProfits'])
            contador = contador + 1
            
        auxVar = {'FitProfit': soma, 'FitQuantities': person}
        arrayFitProfitAndQuantity.append(auxVar)
        arraySomaProfits.append(soma)
        sorted_data = sorted(arrayFitProfitAndQuantity, key=lambda x: x['FitProfit'], reverse=True)
    contador+=1
        
    # array = np.array(arraySomaProfits)
    # array[::-1].sort()
    # print(array)
    
    print("sorted_data",sorted_data)
    print("len",len(sorted_data))
    return sorted_data
lis=createFitnes(population, products)


sorted_data [{'FitProfit': 5900, 'FitQuantities': [10, 9, 10, 10]}, {'FitProfit': 5780, 'FitQuantities': [10, 10, 9, 8]}, {'FitProfit': 5540, 'FitQuantities': [10, 9, 9, 4]}, {'FitProfit': 5330, 'FitQuantities': [9, 7, 10, 3]}, {'FitProfit': 5060, 'FitQuantities': [5, 9, 9, 6]}, {'FitProfit': 5050, 'FitQuantities': [10, 5, 10, 5]}, {'FitProfit': 5020, 'FitQuantities': [9, 10, 7, 2]}, {'FitProfit': 5010, 'FitQuantities': [6, 7, 10, 1]}, {'FitProfit': 4990, 'FitQuantities': [5, 7, 10, 9]}, {'FitProfit': 4940, 'FitQuantities': [4, 9, 9, 4]}, {'FitProfit': 4880, 'FitQuantities': [7, 7, 9, 8]}, {'FitProfit': 4850, 'FitQuantities': [6, 9, 8, 5]}, {'FitProfit': 4810, 'FitQuantities': [6, 6, 10, 1]}, {'FitProfit': 4800, 'FitQuantities': [7, 8, 8, 10]}, {'FitProfit': 4790, 'FitQuantities': [9, 4, 10, 9]}, {'FitProfit': 4680, 'FitQuantities': [1, 9, 9, 8]}, {'FitProfit': 4660, 'FitQuantities': [2, 7, 10, 6]}, {'FitProfit': 4650, 'FitQuantities': [5, 10, 7, 5]}, {'FitProfit': 4630, 'FitQuantities

In [928]:
#saves the first best quantity% to the final list
#finalPopulation = []
def getBestXElementsDic(lis, percentage):
   
    for i in range(len(lis)//percentage):
 
        #returns {'FitProfit': 2850, 'FitQuantities': [5, 4, 5, 5]},...
        finalPopulation.append(lis[i])
        #returns  [5, 4, 5, 5],....
        #finalPopulation.append(lis[i]['FitQuantities'])
       
  

    return finalPopulation

dic=getBestXElementsDic(lis,10)
print("-----------------------------------------finalPopulationList---------------------------------------------------")
print(dic)
print("--------------------------------------------------------------------------------------------------------")


-----------------------------------------finalPopulationList---------------------------------------------------
[{'FitProfit': 5610, 'FitQuantities': [8, 9, 10, 1]}, {'FitProfit': 5590, 'FitQuantities': [5, 10, 10, 9]}, {'FitProfit': 5300, 'FitQuantities': [6, 8, 10, 10]}, {'FitProfit': 5230, 'FitQuantities': [7, 9, 9, 3]}, {'FitProfit': 5110, 'FitQuantities': [9, 6, 10, 1]}, {'FitProfit': 5020, 'FitQuantities': [3, 10, 9, 2]}, {'FitProfit': 4980, 'FitQuantities': [5, 7, 10, 8]}, {'FitProfit': 4980, 'FitQuantities': [3, 8, 10, 8]}, {'FitProfit': 4920, 'FitQuantities': [8, 7, 9, 2]}, {'FitProfit': 4910, 'FitQuantities': [10, 6, 9, 1]}]
--------------------------------------------------------------------------------------------------------


In [929]:
#saves the first best quantity% to the final list
#finalPopulation = []
def getBestXElementsQuantity(lis, percentage):
   
    for i in range(len(lis)//percentage):
 
        #returns {'FitProfit': 2850, 'FitQuantities': [5, 4, 5, 5]},...
        #finalPopulation.append(lis[i])
        #returns  [5, 4, 5, 5],....
        finalPopulation.append(lis[i]['FitQuantities'])
       
  

    return finalPopulation

quantities=getBestXElementsQuantity(lis,10)
print("-----------------------------------------finalPopulationList---------------------------------------------------")
print(quantities)
print("--------------------------------------------------------------------------------------------------------")


-----------------------------------------finalPopulationList---------------------------------------------------
[{'FitProfit': 5610, 'FitQuantities': [8, 9, 10, 1]}, {'FitProfit': 5590, 'FitQuantities': [5, 10, 10, 9]}, {'FitProfit': 5300, 'FitQuantities': [6, 8, 10, 10]}, {'FitProfit': 5230, 'FitQuantities': [7, 9, 9, 3]}, {'FitProfit': 5110, 'FitQuantities': [9, 6, 10, 1]}, {'FitProfit': 5020, 'FitQuantities': [3, 10, 9, 2]}, {'FitProfit': 4980, 'FitQuantities': [5, 7, 10, 8]}, {'FitProfit': 4980, 'FitQuantities': [3, 8, 10, 8]}, {'FitProfit': 4920, 'FitQuantities': [8, 7, 9, 2]}, {'FitProfit': 4910, 'FitQuantities': [10, 6, 9, 1]}, [8, 9, 10, 1], [5, 10, 10, 9], [6, 8, 10, 10], [7, 9, 9, 3], [9, 6, 10, 1], [3, 10, 9, 2], [5, 7, 10, 8], [3, 8, 10, 8], [8, 7, 9, 2], [10, 6, 9, 1]]
--------------------------------------------------------------------------------------------------------


In [930]:
#removes previously saved elements, and gets the list of elements that are going to crossover
def getRemainingForCrossover(lis, percentage):
    crossoverElements = lis[len(lis)//percentage:]
    return crossoverElements

crossoverElements=getRemainingForCrossover(lis,10)
print("--------------------------------------CrossoverList------------------------------------------------------")
print(crossoverElements)
print(len(crossoverElements))
print("--------------------------------------------------------------------------------------------")

###!!!!!!!!!!!!!!!!! return just quantities so that the next method doesn't need to be called

--------------------------------------CrossoverList------------------------------------------------------
[{'FitProfit': 4900, 'FitQuantities': [2, 8, 10, 10]}, {'FitProfit': 4860, 'FitQuantities': [8, 8, 8, 6]}, {'FitProfit': 4650, 'FitQuantities': [3, 8, 9, 5]}, {'FitProfit': 4650, 'FitQuantities': [2, 10, 8, 5]}, {'FitProfit': 4620, 'FitQuantities': [1, 9, 9, 2]}, {'FitProfit': 4610, 'FitQuantities': [7, 6, 9, 1]}, {'FitProfit': 4580, 'FitQuantities': [10, 7, 7, 8]}, {'FitProfit': 4550, 'FitQuantities': [1, 10, 8, 5]}, {'FitProfit': 4520, 'FitQuantities': [3, 6, 10, 2]}, {'FitProfit': 4510, 'FitQuantities': [3, 9, 8, 1]}, {'FitProfit': 4470, 'FitQuantities': [10, 2, 10, 7]}, {'FitProfit': 4440, 'FitQuantities': [2, 9, 8, 4]}, {'FitProfit': 4400, 'FitQuantities': [9, 8, 6, 10]}, {'FitProfit': 4300, 'FitQuantities': [5, 5, 9, 10]}, {'FitProfit': 4280, 'FitQuantities': [3, 9, 7, 8]}, {'FitProfit': 4190, 'FitQuantities': [2, 6, 9, 9]}, {'FitProfit': 4100, 'FitQuantities': [7, 6, 7, 10]}

In [931]:
def getQuantities(crossoverElements):
    
    print(len(crossoverElements))
    fit_quantities = [d['FitQuantities'] for d in crossoverElements]
    return fit_quantities

crossoverElementsQuantities=getQuantities(crossoverElements)

print(crossoverElementsQuantities)

90
[[2, 8, 10, 10], [8, 8, 8, 6], [3, 8, 9, 5], [2, 10, 8, 5], [1, 9, 9, 2], [7, 6, 9, 1], [10, 7, 7, 8], [1, 10, 8, 5], [3, 6, 10, 2], [3, 9, 8, 1], [10, 2, 10, 7], [2, 9, 8, 4], [9, 8, 6, 10], [5, 5, 9, 10], [3, 9, 7, 8], [2, 6, 9, 9], [7, 6, 7, 10], [1, 6, 9, 4], [6, 2, 10, 4], [4, 9, 6, 3], [8, 4, 8, 3], [1, 7, 8, 9], [4, 7, 7, 4], [8, 2, 9, 3], [9, 4, 7, 10], [7, 8, 5, 1], [7, 3, 8, 9], [5, 10, 4, 8], [7, 3, 8, 6], [6, 3, 8, 8], [5, 2, 9, 8], [7, 7, 5, 8], [7, 10, 3, 6], [6, 4, 7, 10], [6, 4, 7, 7], [3, 4, 8, 4], [9, 7, 4, 4], [7, 5, 6, 4], [8, 6, 5, 4], [6, 2, 8, 9], [2, 1, 10, 8], [8, 10, 2, 8], [6, 2, 8, 8], [10, 7, 3, 7], [10, 1, 7, 5], [7, 8, 3, 7], [3, 7, 5, 4], [4, 5, 6, 3], [5, 1, 8, 8], [5, 10, 2, 8], [10, 6, 3, 5], [9, 6, 3, 8], [4, 7, 4, 6], [1, 4, 7, 3], [5, 2, 7, 2], [3, 4, 6, 8], [3, 1, 8, 7], [3, 5, 5, 10], [3, 5, 5, 7], [5, 4, 5, 6], [3, 5, 5, 3], [10, 1, 5, 9], [6, 9, 1, 4], [9, 7, 1, 8], [4, 5, 4, 7], [4, 5, 4, 6], [10, 5, 2, 6], [1, 5, 5, 5], [5, 9, 1, 5], [5, 1

In [932]:

def crossoverMethod(parent1, parent2, crossover_rate):
    if random.uniform(0, 1) < crossover_rate:
        crossover_point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
    else:
        return parent1, parent2



In [933]:
crossover_rate = 0.8 # Crossover rate
percentageCrossover = 10 # Crossover rate
def crossOver(dic):
    listCrossOver=[]
    for gene in dic:
      
        #selects first and last genes to crossover
        parent1 = dic[0]['FitQuantities']
        parent2 = dic[-1]['FitQuantities']

        child1, child2 = crossoverMethod(parent1, parent2, crossover_rate)
        # listCrossOver.append(child1)
        # listCrossOver.append(child2)
        if child1 not in listCrossOver:
            listCrossOver.append(child1)
        if child2 not in listCrossOver:
            listCrossOver.append(child1)
      


    #return first 10% of the genes resulting from the crossover      !!!! deveriam ser os 10% MELHORES-necessário fazer calculo
    #return listCrossOver[:len(listCrossOver)//percentageCrossover]
    return listCrossOver

print(crossOver(crossoverElements))
print(len(crossOver(crossoverElements)))


[[2, 3, 1, 7], [2, 3, 1, 7], [2, 8, 10, 10], [2, 8, 10, 10], [2, 8, 10, 7], [2, 8, 10, 7], [2, 8, 1, 7], [2, 8, 1, 7], [2, 3, 1, 7], [2, 3, 1, 7], [2, 8, 10, 10], [2, 8, 10, 7], [2, 8, 10, 7], [2, 8, 10, 7], [2, 3, 1, 7], [2, 8, 10, 10], [2, 8, 1, 7], [2, 8, 10, 7], [2, 8, 1, 7], [2, 8, 10, 7], [2, 8, 1, 7], [2, 8, 10, 7], [2, 8, 10, 10], [2, 3, 1, 7], [2, 8, 1, 7], [2, 8, 10, 7], [2, 8, 1, 7], [2, 8, 1, 7], [2, 8, 10, 7], [2, 3, 1, 7], [2, 8, 10, 10], [2, 8, 10, 10], [2, 8, 1, 7], [2, 8, 10, 10], [2, 8, 10, 10], [2, 8, 10, 7], [2, 8, 10, 7], [2, 8, 10, 10], [2, 3, 1, 7], [2, 8, 1, 7], [2, 8, 1, 7], [2, 8, 10, 10], [2, 8, 10, 7], [2, 8, 10, 10], [2, 3, 1, 7], [2, 8, 1, 7], [2, 8, 10, 10], [2, 8, 10, 7], [2, 3, 1, 7], [2, 3, 1, 7], [2, 8, 10, 10], [2, 3, 1, 7], [2, 8, 10, 7], [2, 8, 1, 7], [2, 8, 1, 7], [2, 8, 10, 10], [2, 8, 10, 7], [2, 8, 10, 7], [2, 8, 10, 7], [2, 8, 10, 7], [2, 8, 1, 7], [2, 3, 1, 7], [2, 3, 1, 7], [2, 3, 1, 7], [2, 8, 10, 7], [2, 3, 1, 7], [2, 8, 10, 7], [2, 8, 10,

In [12]:
# Create Fitness function to give the best solution-HUGO

def createFitnessFunction(population, products):

    
    arraySomaProfits = []
    arrayFitProfitAndQuantity = []


    # implementar o ciclo para n vezes
    for person in population:

        soma = 0
        contador = 0

        for productQuantity in person:

            soma = soma + (productQuantity * products[contador]['predictedProfits'])
            contador = contador + 1

        auxVar = {'FitProfit': soma, 'FitQuantities': person}
        arrayFitProfitAndQuantity.append(auxVar)
        arraySomaProfits.append(soma)

    array = np.array(arraySomaProfits)
    array[::-1].sort()

    # get out taxNoCrossover to final population
    numberElementsNoCrossover = len(population) * taxNoCrossover
  
    # for i in range(round(numberElementsNoCrossover)):

    #     for element in arrayFitProfitAndQuantity:

    #         if element['FitProfit'] == array[i]:
    #             finalPopulation.append(element['FitQuantities'])
    #             population.remove(element['FitQuantities'])

    # return array


    # print("LenBEfore",len(arrayFitProfitAndQuantity))
    # print("LenBEforePOP",len(population))
    # print("LenBEforeFINAL",len(finalPopulation))
    for dic1,dic2 in zip(arrayFitProfitAndQuantity[::2], arrayFitProfitAndQuantity[1::2]):
            if random.randint(0, 100) > 90:
                arrayFitProfitAndQuantity.remove(dic1)
                arrayFitProfitAndQuantity.remove(dic2)
                # print("Dic1",dic1['FitQuantities'])
                # print("Dic2",dic2['FitQuantities'])

                # removes from population list elements (dic1 and dic2) that are not passing to teh crossover
                population[:] = [x for x in population if x != dic1['FitQuantities'] and x != dic2['FitQuantities']]
                finalPopulation.append(dic1['FitQuantities'])
                finalPopulation.append(dic2['FitQuantities'])  

                # print("LENDURING",len(arrayFitProfitAndQuantity))

            
    
    # print("LENarrayFitProfitAndQuantity",len(arrayFitProfitAndQuantity))
    # print("LenAfterPOP",len(population))
 
    # print("LenFInalPopulation",len(finalPopulation))

    print("finalPopulation",finalPopulation)
    print("lenPop",len(population) )
    #population= genes que vao fazer o crossover
    print("population",population)

    return array

#fitSolutions = createFitnessFunction(population, products)
#maxFitSolution = np.max(createFitnessFunction(population, products))

#print(fitSolutions)
print(finalPopulation)
print("len",len(population) )
#population= genes que vao fazer o crossover
print(population)

[]
len 0
[]


In [1120]:

#---------------------------------UNUSED-----------------------------------------------
# Create Fitness function to give the best solution

arraySomaProfits = []
arrayFitProfitAndQuantity = list
finalPopulation = []

# if this element is 1 the first tax of best individuos is added to final population
elementToGetFirstBestIndividuos = 0

taxNoCrossover = 0.10
def createFitnessction(population, products):
 for person in population:
        
        soma = 0
        contador = 0
        
        for productQuantity in person:
            
            soma = soma + (productQuantity * products[contador]['predictedProfits'])
            contador = contador + 1
            
        auxVar = {'FitProfit': soma, 'FitQuantities': person}
        arrayFitProfitAndQuantity.insert(auxVar)
        arraySomaProfits.append(soma)
        # array = np.array(arraySomaProfits)
        # array[::-1].sort()

 sorted_elements = sorted(arrayFitProfitAndQuantity, key=lambda x: x['FitProfit'], reverse=True)


 return sorted_elements 

print(createFitnessction(population, products))

TypeError: descriptor 'insert' for 'list' objects doesn't apply to a 'dict' object

In [13]:
# Function to crossover

def crossover(population):
    print(len(population))
    while len(population)>0:

        if len(population) == 1:
            finalPopulation.append(population[0])
            population.remove(population[0])
        else:
            firstPai = population[0]
            secondPai = population[1]
            crossoverPoint = random.randint(0, len(population[0]))
            for i in range(crossoverPoint):
                aux = firstPai[i]
                firstPai[i] = secondPai[i]
                secondPai[i] = aux

             #validate that first chield is good
            totalAmountRestriction1 = validateTotalAmount(firstPai)
            totalQuantityRestriction1 = validateTotalQuantity(firstPai)
            dimensionRestriction1 = validateDimension(firstPai, products)

            #validate that second chield is good
            totalAmountRestriction2 = validateTotalAmount(secondPai)
            totalQuantityRestriction2 = validateTotalQuantity(secondPai)
            dimensionRestriction2 = validateDimension(secondPai, products)
                  
            if (totalAmountRestriction1 == 1) and (totalQuantityRestriction1 == 1) and (dimensionRestriction1 == 1) and (totalAmountRestriction2 == 1) and (totalQuantityRestriction2 == 1) and (dimensionRestriction2 == 1):
                
                population.remove(population[0])
                population.remove(population[0])
                finalPopulation.append(firstPai)
                finalPopulation.append(secondPai)
            

    #crossover(population)
    print("len(population)",len(population))
    print("population",population)
    print("---------------------------------------FINAL---------------------------------------------")
    print("finalPopulation",finalPopulation)
    print("len(finalPopulation)",len(finalPopulation))

In [14]:
taxMutation = 5
changeGene = 0.1

def mutation(finalPopulation):
    mutationList=finalPopulation[:]
    print("LENBEfore",len(mutationList))
    for x in mutationList:
        t=random.randint(0, 100)
        print(t)
        if t >= taxMutation:
            mutationList.remove(x)
            print("LENDURING",len(mutationList))

    print("LENBAfter",len(mutationList))

    #mutationList=len(finalPopulation)*taxMutation
    print(mutationList)
    i = 0

    # for i in range(round(len(mutationList))):

    #     #index where the mutation is going to be applied
        
    #     lenFinalPopulation = len(finalPopulation[i])-1

    #     index=random.randint(0, lenFinalPopulation)
    #     print("Para alterar", finalPopulation[i][index])
    #     print("Quanto alterar", round(finalPopulation[i][index]*changeGene))

    #     finalPopulation[i][index] = finalPopulation[i][index] + round(finalPopulation[i][index]*changeGene)
    #     print("Alterado", finalPopulation[i][index])
            
    #     totalAmountRestriction = validateTotalAmount(finalPopulation[i])
    #     totalQuantityRestriction = validateTotalQuantity(finalPopulation[i])
    #     dimensionRestriction = validateDimension(finalPopulation[i], products)
              
    #     if (totalAmountRestriction == 0) or (totalQuantityRestriction == 0) or (dimensionRestriction == 0):
    #         print("Removido", finalPopulation[i])
    #         finalPopulation.remove(finalPopulation[i])
    print("TamanhoFinalPopulationBefore",len(finalPopulation))   
    for el in finalPopulation:
        initialEl=el
        for l in mutationList:
            #find genes that are going to mutate
            if el==l:
                index=random.randint(0, len(mutationList[0])-1)
                print(" index",index)
                print("Para alterar",el)
                print("Quanto alterar", round(el[index]))

                el[index] = el[index] + round(el[index]*changeGene)
                print("Alterado", el[index])
                        
                totalAmountRestriction = validateTotalAmount(el)
                totalQuantityRestriction = validateTotalQuantity(el)
                dimensionRestriction = validateDimension(el, products)
                        
                if (totalAmountRestriction == 0) or (totalQuantityRestriction == 0) or (dimensionRestriction == 0):
                    print("Não mutado", finalPopulation[i])
                    el=initialEl

    print("len(finalPopulation[0])",len(finalPopulation[0]))
    print("finalPopulation",finalPopulation)
    print("TamanhoFinalPopulationAfter",len(finalPopulation))
    print(finalPopulation[0][0])

In [1131]:
print(len(finalPopulation[0]))
mutation(finalPopulation)
print("FINAL",finalPopulation)
print("TamanhoFinalPopulationAfter",len(finalPopulation))
print(finalPopulation[0][0])

4
LENBEfore 100
27
LENDURING 99
35
LENDURING 98
78
LENDURING 97
89
LENDURING 96
71
LENDURING 95
56
LENDURING 94
29
LENDURING 93
2
37
LENDURING 92
71
LENDURING 91
60
LENDURING 90
9
LENDURING 89
61
LENDURING 88
6
LENDURING 87
11
LENDURING 86
93
LENDURING 85
83
LENDURING 84
39
LENDURING 83
90
LENDURING 82
10
LENDURING 81
40
LENDURING 80
87
LENDURING 79
86
LENDURING 78
39
LENDURING 77
67
LENDURING 76
13
LENDURING 75
31
LENDURING 74
96
LENDURING 73
29
LENDURING 72
1
23
LENDURING 71
52
LENDURING 70
13
LENDURING 69
5
LENDURING 68
87
LENDURING 67
96
LENDURING 66
87
LENDURING 65
84
LENDURING 64
81
LENDURING 63
65
LENDURING 62
78
LENDURING 61
35
LENDURING 60
17
LENDURING 59
43
LENDURING 58
46
LENDURING 57
29
LENDURING 56
66
LENDURING 55
59
LENDURING 54
36
LENDURING 53
17
LENDURING 52
48
LENDURING 51
LENBAfter 51
[[4, 9, 10, 1], [9, 1, 10, 4], [2, 8, 10, 9], [4, 1, 9, 10], [9, 3, 3, 6], [10, 6, 7, 3], [1, 3, 9, 1], [1, 2, 3, 8], [7, 4, 4, 8], [5, 10, 8, 2], [8, 8, 9, 1], [5, 4, 8, 6], [9, 10, 7, 

In [ ]:
print(len(finalPopulation))

98


In [1]:

population = []
populationSize=100
finalPopulation = []
taxNoCrossover = 10


# Functions to validate restrictions
# return 1 if restriction is repected else 0

#Total Amount Restriction
def validateTotalAmount(individuo):
    somaTotalAmountIndividuo = sum(individuo)
    if (somaTotalAmountIndividuo >= minTotalAmount) and (somaTotalAmountIndividuo <= maxTotalAmount):
        return 1
    else:
        return 0
    
#Total Quantity Restriction
def validateTotalQuantity(individuo):
    contador = 0
    individuoLen = len(individuo)
    for indAux in individuo:
        if (indAux >= minAmountProduct) and (indAux <= maxAmountProduct):
            contador = contador + 1
    if(contador == individuoLen):
        return 1
    else:
        return 0

#Dimension Restriction
def validateDimension(individuo, products):
    totalDimension = 0
    contador = 0
    for indAux in individuo:
        totalDimension = indAux * products[contador]['dimension']
        contador = contador + 1
    if(totalDimension > storeDimension):
        return 0
    else:
        return 1


#-------------------------------------------------------------------------------------------------------------------
# Create First Population

def firstpopulation(products):

    while len(population) < populationSize:
        solution = []
        totalQuantity = 0
        totalDimension = 0
        
        #generates the chromossomes each representing one product
        for j in range(len(products)):
            
            quantity = random.randint(0, maxAmountProduct)
            solution.append(quantity)
            
        #Total Amount Restriction
        totalAmountRestriction = validateTotalAmount(solution)
        #Total Quantity Restriction
        totalQuantityRestriction = validateTotalQuantity(solution)
        #Dimension Restriction
        dimensionRestriction = validateDimension(solution, products)
        
        if (totalAmountRestriction == 1) and (totalQuantityRestriction == 1) and (dimensionRestriction == 1):
            population.append(solution)
            

    print(len(firstpopulation(products)))
    print(population)
    
    return population
#-------------------------------------------------------------------------------------------------------------------


# Create Fitness function to give the best solution-HUGO

def createFitnessFunction(population, products):

    
    arraySomaProfits = []
    arrayFitProfitAndQuantity = []


    # implementar o ciclo para n vezes
    for person in population:

        soma = 0
        contador = 0

        for productQuantity in person:

            soma = soma + (productQuantity * products[contador]['predictedProfits'])
            contador = contador + 1

        auxVar = {'FitProfit': soma, 'FitQuantities': person}
        arrayFitProfitAndQuantity.append(auxVar)
        arraySomaProfits.append(soma)

    array = np.array(arraySomaProfits)
    array[::-1].sort()

    # get out taxNoCrossover to final population
    numberElementsNoCrossover = len(population) * taxNoCrossover
  
    # for i in range(round(numberElementsNoCrossover)):

    #     for element in arrayFitProfitAndQuantity:

    #         if element['FitProfit'] == array[i]:
    #             finalPopulation.append(element['FitQuantities'])
    #             population.remove(element['FitQuantities'])

    # return array


    # print("LenBEfore",len(arrayFitProfitAndQuantity))
    # print("LenBEforePOP",len(population))
    # print("LenBEforeFINAL",len(finalPopulation))
    for dic1,dic2 in zip(arrayFitProfitAndQuantity[::2], arrayFitProfitAndQuantity[1::2]):
            if random.randint(0, 100) > 90:
                arrayFitProfitAndQuantity.remove(dic1)
                arrayFitProfitAndQuantity.remove(dic2)
                # print("Dic1",dic1['FitQuantities'])
                # print("Dic2",dic2['FitQuantities'])

                # removes from population list elements (dic1 and dic2) that are not passing to teh crossover
                population[:] = [x for x in population if x != dic1['FitQuantities'] and x != dic2['FitQuantities']]
                finalPopulation.append(dic1['FitQuantities'])
                finalPopulation.append(dic2['FitQuantities'])  

                # print("LENDURING",len(arrayFitProfitAndQuantity))

            
    
    # print("LENarrayFitProfitAndQuantity",len(arrayFitProfitAndQuantity))
    # print("LenAfterPOP",len(population))
 
    # print("LenFInalPopulation",len(finalPopulation))

    print("finalPopulation",finalPopulation)
    print("lenPop",len(population) )
    #population= genes que vao fazer o crossover
    print("population",population)

    return array

#-------------------------------------------------------------------------------------------------------------------


# Function to crossover

def crossover(population):
    print(len(population))
    while len(population)>0:

        if len(population) == 1:
            finalPopulation.append(population[0])
            population.remove(population[0])
        else:
            firstPai = population[0]
            secondPai = population[1]
            crossoverPoint = random.randint(0, len(population[0]))
            for i in range(crossoverPoint):
                aux = firstPai[i]
                firstPai[i] = secondPai[i]
                secondPai[i] = aux

             #validate that first chield is good
            totalAmountRestriction1 = validateTotalAmount(firstPai)
            totalQuantityRestriction1 = validateTotalQuantity(firstPai)
            dimensionRestriction1 = validateDimension(firstPai, products)

            #validate that second chield is good
            totalAmountRestriction2 = validateTotalAmount(secondPai)
            totalQuantityRestriction2 = validateTotalQuantity(secondPai)
            dimensionRestriction2 = validateDimension(secondPai, products)
                  
            if (totalAmountRestriction1 == 1) and (totalQuantityRestriction1 == 1) and (dimensionRestriction1 == 1) and (totalAmountRestriction2 == 1) and (totalQuantityRestriction2 == 1) and (dimensionRestriction2 == 1):
                
                population.remove(population[0])
                population.remove(population[0])
                finalPopulation.append(firstPai)
                finalPopulation.append(secondPai)
            

    #crossover(population)
    print("len(population)",len(population))
    print("population",population)
    print("---------------------------------------FINAL---------------------------------------------")
    print("finalPopulation",finalPopulation)
    print("len(finalPopulation)",len(finalPopulation))

#-------------------------------------------------------------------------------------------------------------------
taxMutation = 5
changeGene = 0.1

def mutation(finalPopulation):
    mutationList=finalPopulation[:]
    print("LENBEfore",len(mutationList))
    for x in mutationList:
        t=random.randint(0, 100)
        print(t)
        if t >= taxMutation:
            mutationList.remove(x)
            print("LENDURING",len(mutationList))

    print("LENBAfter",len(mutationList))

    #mutationList=len(finalPopulation)*taxMutation
    print(mutationList)
    i = 0

    # for i in range(round(len(mutationList))):

    #     #index where the mutation is going to be applied
        
    #     lenFinalPopulation = len(finalPopulation[i])-1

    #     index=random.randint(0, lenFinalPopulation)
    #     print("Para alterar", finalPopulation[i][index])
    #     print("Quanto alterar", round(finalPopulation[i][index]*changeGene))

    #     finalPopulation[i][index] = finalPopulation[i][index] + round(finalPopulation[i][index]*changeGene)
    #     print("Alterado", finalPopulation[i][index])
            
    #     totalAmountRestriction = validateTotalAmount(finalPopulation[i])
    #     totalQuantityRestriction = validateTotalQuantity(finalPopulation[i])
    #     dimensionRestriction = validateDimension(finalPopulation[i], products)
              
    #     if (totalAmountRestriction == 0) or (totalQuantityRestriction == 0) or (dimensionRestriction == 0):
    #         print("Removido", finalPopulation[i])
    #         finalPopulation.remove(finalPopulation[i])
    print("TamanhoFinalPopulationBefore",len(finalPopulation))   
    for el in finalPopulation:
        initialEl=el
        for l in mutationList:
            #find genes that are going to mutate
            if el==l:
                index=random.randint(0, len(mutationList[0])-1)
                print(" index",index)
                print("Para alterar",el)
                print("Quanto alterar", round(el[index]))

                el[index] = el[index] + round(el[index]*changeGene)
                print("Alterado", el[index])
                        
                totalAmountRestriction = validateTotalAmount(el)
                totalQuantityRestriction = validateTotalQuantity(el)
                dimensionRestriction = validateDimension(el, products)
                        
                if (totalAmountRestriction == 0) or (totalQuantityRestriction == 0) or (dimensionRestriction == 0):
                    print("Não mutado", finalPopulation[i])
                    el=initialEl

    print("len(finalPopulation[0])",len(finalPopulation[0]))
    print("finalPopulation",finalPopulation)
    print("TamanhoFinalPopulationAfter",len(finalPopulation))
    print(finalPopulation[0][0])

In [ ]:
print("--------------------------------FIRSTPOP---------------------------")
firstpopulation(products)

In [ ]:
print("--------------------------------Fitness---------------------------")
fitSolutions = createFitnessFunction(population, products)
print("fitSolutions",fitSolutions)

In [ ]:
print("--------------------------------CrossOver---------------------------")
crossover(population)

In [ ]:
print("--------------------------------Mutation---------------------------")
mutation(finalPopulation)

In [7]:


# def geneticAlgorithm():

print("--------------------------------FIRSTPOP---------------------------")
firstpopulation(products)
print("--------------------------------Fitness---------------------------")
fitSolutions = createFitnessFunction(population, products)
print("fitSolutions",fitSolutions)
print("--------------------------------CrossOver---------------------------")
crossover(population)
print("--------------------------------Mutation---------------------------")
mutation(finalPopulation)

    

: 

: 

In [ ]:
geneticAlgorithm()

In [ ]:
import random


def optimize_product_quantities(productsList, maxTotalQuantity, maxProductQuantity, storeDimension):
    # products is a list of tuples, where each tuple contains the following information about a product:
    # (name, quantity, price, dimension)

    # Create a population of random solutions

    # population = []
    # for i in range(100):
    #     solution = []
    #     totalQuantity = 0
    #     totalDimension = 0
    #     for j in range(len(productsList)):
    #         quantity = random.randint(0, maxProductQuantity)
    #         totalQuantity += quantity
    #         totalDimension += quantity * productsList[j][2]
    #         solution.append(quantity)
    #     if totalQuantity <= maxTotalQuantity and totalDimension <= storeDimension:
    #         population.append(solution)

    #represents code above
    firstpopulation(productsList)

    # Set the number of generations and the mutation rate
    numGenerations = 1000
    mutationRate = 0.01

    # Run the genetic algorithm
    for generation in range(numGenerations):
        # Calculate the fitness of each solution in the population
        populationFitness = []
        for i in range(len(population)):
            totalProfit = 0
            totalQuantity = 0
            totalDimension = 0
            for j in range(len(productsList)):
                totalProfit += population[i][j] * productsList[j][2]
                totalQuantity += population[i][j]
                totalDimension += population[i][j] * productsList[j][3]
            if totalQuantity <= maxTotalQuantity and totalDimension <= storeDimension:
                populationFitness.append(totalProfit)
            else:
                populationFitness.append(0)

        # Select the fittest solutions for breeding
        fittestSolutions = []
        for i in range(int(len(population) / 2)):
            fittestIndex = populationFitness.index(max(populationFitness))
            fittestSolutions.append(population[fittestIndex])
            populationFitness[fittestIndex] = 0
        population = fittestSolutions

        # Breed the solutions to create a new population
        newPopulation = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[random.randint(0, len(population) - 1)]
            child = []
            for j in range(len(productsList)):
                if random.random() < 0.5:
                    child.append(parent1[j])
                else:
                    child.append(parent2[j])

            # Mutate the child
            if random.random() < mutationRate:
                mutationIndex = random.randint(0, len(productsList) - 1)
                child[mutationIndex] = random.randint(0, maxProductQuantity)

            newPopulation.append(child)
        population = newPopulation
        print(generation)
    # Return the fittest solution

    return population[0]

In [ ]:


# Define the fitness function
def fitness(x):
    # x is the number of products to buy
    # We can define the fitness as the difference between the desired stock level and the actual stock level
    # The desired stock level is 10
    return abs(10 - x)

# Initialize the population with random solutions
population = [random.randint(0, 20) for _ in range(10)]

# Set the maximum number of generations
max_generations = 100

# Iterate over the generations
for generation in range(max_generations):
    # Evaluate the fitness of each solution
    fitness_values = [fitness(x) for x in population]
    
    # Select the best solutions as parents
    # We'll use tournament selection here, which involves selecting a random subset of the population
    # and choosing the best solution from that subset as the parent
    parent1 = min(random.sample(population, 5), key=fitness)
    parent2 = min(random.sample(population, 5), key=fitness)
    
    # Create offspring by combining the parents using crossover
    # We'll use one-point crossover here, which involves selecting a random point in the parents' genomes
    # and swapping the genes on either side of the point to create the offspring
    crossover_point = random.randint(1, len(parent1))
    offspring = parent1[:crossover_point] + parent2[crossover_point:]
    
    # Add some randomness to the offspring using mutation
    # We'll flip a random bit in the offspring's genome
    mutation_point = random.randint(0, len(offspring) - 1)
    offspring[mutation_point] = 1 - offspring[mutation_point]
    
    # Replace the worst solution in the population with the offspring
    worst_solution = max(zip(population, fitness_values), key=lambda x: x[1])
    population[population.index(worst_solution[0])] = offspring

# The final solution is the best solution in the final generation
best_solution = min(zip(population, fitness_values), key=lambda x: x[1])
print(f"The optimal number of products to buy is: {best_solution[0]}")

TypeError: object of type 'int' has no len()

In [ ]:

  # evaluate the fitness of a given solution
  def evaluate_fitness(products):

#  # check if better (e.g. perform a tournament)
#  if scores[ix] < scores[selection_ix]:
#  selection_ix = ix
#  return pop[selection_ix]
#{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0},  

    bestQuantity = 0
        if products['predictedProfits'] < products['predictedProductQuantities']:
      return 0

    
    total_dimension = sum([q * d for q, d in zip(solution, product_dimensions)])
    if total_quantity < min_quantity or total_quantity > max_quantity or total_dimension > store_dimension:
      return 0
    profit = calculate_profit(solution)
    # calculate the deviation from the target profit
    deviation = abs(profit - target_profit)
    # the fitness is higher for solutions that have a lower deviation from the target profit
    return 1 / (deviation + 1)

  # genetic algorithm function
  def genetic_algorithm(fitness_fn):
    # initialize population
    population = initialize_population(product_quantities)
    for generation in range(max_generations):
      # evaluate fitness
      fitnesses = [fitness_fn(s) for s in population]
      # select parents
      parents = select_parents(population, fitnesses)
      # create offspring
      offspring = create_offspring(parents)
      # mutate offspring
      offspring = mutate(offspring)
      # replace least fit individuals in population with offspring
      population = replace_least_fit(population, offspring)
    # return the fittest solution
    return population[0]

  # use the genetic algorithm to find the optimal solution
  return genetic_algorithm(evaluate_fitness)

In [ ]:
#--------------------------------------------------------------------------
